In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

In [ ]:
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder # 카테고리형 데이터를 수치형 데이터로 변환

In [ ]:
cd /content/drive/MyDrive/AI/AICE/Tabular


In [ ]:
data = pd.read_csv('Airline.csv')
data.head()

##라벨 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['satisfaction'] = encoder.fit_transform(data['satisfaction'])
encoder.classes_

In [ ]:
data.head()

##결측치 확인

In [ ]:
def check_missing_col(dataframe):
  missing_col = []
  for col in dataframe.columns:
    missing_values = sum(dataframe[col].isna()) # isna() : 데이터프레임 내에 결측 값을 확인하여, 그 결과를 True 또는 False로 반환
    is_missing = True if missing_values >= 1 else False
    if is_missing:
      print(f'결측치가 있는 컬럼은: {col} 입니다')
      print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
      missing_col.append([col, dataframe[col].dtype])
  if missing_col == []:
    print("결측치가 존재하지 않습니다")
  return missing_col

missing_col = check_missing_col(data)

In [ ]:
for col in data.columns:
    msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (data[col].isnull().sum() / data[col].shape[0]))
    print(msg)

####결측치를 중앙값으로 채우기

In [ ]:
#결측치를 중앙값으로 채우기
#data["Arrival Delay in Minutes"].fillna(data["Arrival Delay in Minutes"].median(), inplace=True)
#결측치를 특정값으로
#data["Arrival Delay in Minutes"].fillna(0, inplace=True)
#결측치를 평균값으로
#data["Arrival Delay in Minutes"].fillna(data["Arrival Delay in Minutes"].mean(), inplace=True)
#결측치를 최빈값으로
#data["Arrival Delay in Minutes"].fillna(data["Arrival Delay in Minutes"].mode()[0], inplace=True)
#다른 컬럼의 값으로
data["Arrival Delay in Minutes"].fillna(data["Departure Delay in Minutes"], inplace=True)

In [ ]:
data.describe()

###Feature 분포 시각화

In [ ]:
plt.style.use("ggplot") # matplotlib stylesheet

# 히스토그램을 사용해서 데이터의 분포를 확인
plt.figure(figsize=(25,20))
plt.suptitle("Data Histogram", fontsize=40)

# id는 제외
cols = data.columns
for i in range(len(cols)):
  plt.subplot(5,5,i+1)
  plt.title(cols[i], fontsize = 20)
  if len(data[cols[i]].unique()) > 20: # 유일 값 개수 임의의 20개로 설정 -> Age, Flight Distance, Departure Delay in Minutes, Arrival Delay in Minutes 표현
    plt.hist(data[cols[i]], bins=20, color='b', alpha=0.7) # bins - 가로축 구간의 개수, color - 색, alpha - 투명도(0-1)
  else:
    temp = data[cols[i]].value_counts() # temp(key value Series)
    plt.bar(temp.keys(), temp.values, width=0.5, alpha=0.7)
    plt.xticks(temp.keys())
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # 여백 설정
plt.show()

### SATISFACTION과 Feature의 관계

In [ ]:
target = "satisfaction"

# 범주형 데이터 분리
categorical_feature = data.columns[data.dtypes =='object']

plt.figure(figsize=(20,15))
plt.suptitle("Violin Plot", fontsize=40)

for i in range(len(categorical_feature)):
  plt.subplot(2,2,i+1)
  plt.xlabel(categorical_feature[i])
  plt.ylabel(target)
  sns.violinplot(x=data[categorical_feature[i]], y=data[target])
plt.tight_layout(rect=[0,0.03,1,0.95])
plt.show()

In [ ]:
# 수치형 데이터 분리
numeric_feature = data.columns[(data.dtypes=='int64') | (data.dtypes=='float')]
num_data = data[numeric_feature]

# 박스 플롯
fig, axes = plt.subplots(3,6,figsize=(25,20))

fig.suptitle('Box Plot', fontsize=40)
for ax, col in zip(axes.flat, num_data.columns[:-1]): # axes.flat : 각 하위 plot / num_data.columns[:-1] : 각각의 columns
  sns.boxplot(x= 'satisfaction', y=col, ax=ax, data=num_data)
  ax.set_title(col, fontsize=20)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 상관관계 시각화

In [ ]:
# 먼저 상관계수 계산을 위해서는 텍스트 형식(카테고리형 데이터)의 데이터를 숫자로 변환 HOW? labelencoder를 사용하면 범주형 데이터를 수치형 데이터로 변환가능
corr_df = data.copy()
corr_df[corr_df.columns[corr_df.dtypes=='O']] = corr_df[corr_df.columns[corr_df.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)
# 상관관계 분석도
plt.figure(figsize=(35,25))

heat_table = corr_df.corr()
mask = np.zeros_like(heat_table) # np.zeros_like : 입력값에 맞는 모양의 0으로 가득찬 array 배출
mask[np.triu_indices_from(mask)] = True # upper-triangle of arr
heatmap_ax = sns.heatmap(heat_table, annot=True, mask = mask, cmap='coolwarm') # annot : 각 셀의 값 표현
heatmap_ax.set_xticklabels(heatmap_ax.get_xticklabels(), fontsize=15)
heatmap_ax.set_yticklabels(heatmap_ax.get_yticklabels(), fontsize=15)
plt.title("correlation between features", fontsize = 40)
plt.show()

### 위 그래프 분석
+다중공선성 확인 : 두 계수의 상관계숙 0.7이상인 것 찾기
  - Departure Delay in Minutes, Arrival Delay in Minutes
     . 둘 중 satisfaction과 상관계수가 더 낮은 것 삭제(Departure)
  - Food and Drink, Seat comfort
     . 둘 중 satisfaction과 상관계수가 더 낮은 것 삭제(Food and Drink)

In [ ]:
#Departure, Food and Drink 삭제
data.drop(columns=['Departure Delay in Minutes', 'Food and drink'], axis =1 , inplace=True)


In [ ]:
data.describe()

###이상치 확인

In [ ]:
data.plot(kind="box", subplots=True, layout=(5,5), figsize=(15,21))
plt.show()

+Flight Distance, On-board service, Checkin service, Arrival Delay in Minutes
에서 이상치 확인,

###이상치의 인덱스를 구하는 함수



In [ ]:
def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25, 75])
    # 넘파이의 값을 퍼센트로 표시해주는 함수

    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)

    return np.where((data > upper_bound) | (data < lower_bound))

In [ ]:
ArrayDelay_index = outliers_iqr(data['Arrival Delay in Minutes'])[0]
Checkin_index = outliers_iqr(data['Checkin service'])[0]
FlightDistance_index = outliers_iqr(data['Flight Distance'])[0]
Onboard_index = outliers_iqr(data['On-board service'])[0]

In [ ]:
len(ArrayDelay_index)/len(data), len(Checkin_index)/len(data), len(FlightDistance_index)/len(data), len(Onboard_index)/len(data)


In [ ]:
#이상치가 적으면 제거, 많으면 평균값으로 대체?
data.loc[ArrayDelay_index, 'Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].mean()
data.loc[FlightDistance_index, 'Flight Distance'] = data['Flight Distance'].mean()
data.loc[Checkin_index, 'Checkin service'] = data['Checkin service'].mean()
data.loc[Onboard_index, 'On-board service'] = data['On-board service'].mean()


In [ ]:
data.plot(kind="box", subplots=True, layout=(5,5), figsize=(15,21))
plt.show()

##데이터 전처리(결측치 처리 후 전처리)

###특징이 많으면 과적합일 발생함, 상관계수가 낮은 특징을 제거

In [ ]:
data.drop('Departure/Arrival time convenient', axis = 1, inplace = True)
data.drop('Gate location', axis = 1, inplace = True)
data.drop('Flight Distance', axis = 1, inplace = True)


In [ ]:
data.shape

###범주형 데이터를 숫자로 변환

In [ ]:
data[data.columns[data.dtypes=='O']] = data[data.columns[data.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)

In [ ]:
data.head()

###전처리가 끝나면 train과 test 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
train_df.shape

#모델 학습

In [ ]:
!pip install catboost

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
def ACCURACY(true, pred):
  score = np.mean(true==pred)
  return score

###값의 범위 차이가 큰 특징들 age / (Departure Delay in Minutes, Arrival Delay in Minutes)로 인해 진행한 결과와 스케일링하지 않은 결과를 비교하자

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler # MinMaxScale 사용

scale_feature = [
    'Age',
    'Arrival Delay in Minutes'
]

def kfold(model, train, scale = False):
    cv_accuracy = []
    cv = StratifiedKFold(n_splits=5)

    n_iter = 0

    for t, v in cv.split(train, train['satisfaction']):

        train_cv = train.iloc[t] # 훈련용
        val_cv = train.iloc[v] # 검증용 분리

        train_X = train_cv.drop('satisfaction', axis=1)
        train_y = train_cv['satisfaction']

        val_X = val_cv.drop('satisfaction', axis=1)
        val_y = val_cv['satisfaction']

        if scale:
            scaler = MinMaxScaler()
            for feature in scale_feature:
                train_X = scaler.fit_transform(train_X)
                val_X = scaler.fit_transform(val_X)

        model.fit(train_X, train_y)
        score = ACCURACY(val_y, model.predict(val_X))

        cv_accuracy.append(score)
        n_iter += 1
    return np.mean(cv_accuracy)

In [ ]:
models = [
 #   KNeighborsClassifier(),
  #  LogisticRegression(),
  #  DecisionTreeClassifier(),
    RandomForestClassifier(),
  #  GradientBoostingClassifier(),
  #  XGBClassifier(eval_metric='logloss'), # eval_metric='logloss' 버전 경고 문구 제거
  #  LGBMClassifier(),
    CatBoostClassifier(silent = True),  # silent = True 학습 내용 출력 제거
   # ExtraTreesClassifier()
]

#print('스케일링 적용 전')
#for model in models:
#    print(f'{type(model).__name__} score: {kfold(model, train_df)}')

#print('스케일링 적용 후')
#for model in models:
#    print(f'{type(model).__name__} score: {kfold(model, train_df, True)}')

##스케일링 적용 전

+KNeighborsClassifier score: 0.8775133142522595
+LogisticRegression score: 0.8222534372705281
+DecisionTreeClassifier score: 0.9297373341212364
+RandomForestClassifier score: 0.9506577444221517
+GradientBoostingClassifier score: 0.9202120561019671
+XGBClassifier score: 0.9517356677029369
+LGBMClassifier score: 0.9474789915712233
+CatBoostClassifier score: 0.9532095416547595
+ExtraTreesClassifier score: 0.950690742277291
스케일링 적용 후
KNeighborsClassifier score: 0.9136785473220528
LogisticRegression score: 0.8310528102628894
DecisionTreeClassifier score: 0.9180562119599489
RandomForestClassifier score: 0.9475339686400368
GradientBoostingClassifier score: 0.9187271514109172
XGBClassifier score: 0.9456530963410827
LGBMClassifier score: 0.9445971861477046
CatBoostClassifier score: 0.9502397564681833
ExtraTreesClassifier score: 0.9487658825163606

###트레인데이터와 테스트데이터 준비

In [ ]:
train_x = train_df.drop('satisfaction', axis=1)
train_y = train_df['satisfaction']

test_x = test_df.drop('satisfaction', axis=1)
test_y = test_df['satisfaction']



In [ ]:
test_x

##모델 예측

In [ ]:
model_RFC = RandomForestClassifier().fit(train_x, train_y)
pred_RFC = model_RFC.predict(test_x)

In [ ]:
df = pd.DataFrame()
df['predict'] = pred_RFC
df['true'] = test_y.values

In [ ]:
np.sum(df['true'] == df['predict'])/len(df)

In [ ]:
#CatboostClassifier
model_CAT = CatBoostClassifier(silent = True).fit(train_x, train_y)
pred_CAT = model_CAT.predict(test_x)
df = pd.DataFrame()
df['predict'] = pred_CAT
df['true'] = test_y.values
np.sum(df['true'] == df['predict'])/len(df)

In [ ]:
test_x.insert(0, 'satisfaction', pred_CAT)


In [ ]:
test_x = test_x.drop('satisfaction', axis=1)

In [ ]:
test_x['satisfaction'] = pred_CAT

In [ ]:
test_x.to_csv('result_with_satisfaction.csv', index=False)  # index=False로 설정하여 인덱스를 저장하지 않습니다.


###최적화 GridSearchCV
####CatBoostClassifier와 RandomForest만

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

In [ ]:
##RandomForest
params = { 'n_estimators' : [10, 100, 1000],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }